In [4]:
"""

Draft.

Train/test split by time:
train [2017-11-06, 2017-11-09]
test  [2017-11-10]
"""

import os
import random
import warnings

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
os.environ['KERAS_BACKEND'] = 'tensorflow'

import pandas as pd
import numpy as np
import keras

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

SEED=42

np.random.seed(SEED)
random.seed(SEED)

In [5]:
dtypes = {
    'ip':'uint32',
    'app': 'uint16',
    'device': 'uint16',
    'os': 'uint16',
    'channel': 'uint16',
    'is_attributed': 'uint8'
}   

def add_mean(df_train, df_val, df_test=None):
    df1 = df_train[['is_attributed', 'app']].groupby(['app']).median().rename(columns={'is_attributed': 'mean1'}).reset_index()
    df_train = pd.merge(df_train, df1, on=['app'], how='left')
    df_val = pd.merge(df_val, df1, on=['app'], how='left')
    #df_test = pd.merge(df_test, df1, on=['app'], how='left')
    
    df2 = df_train[['is_attributed', 'app', 'channel']].groupby(['app', 'channel']).median().rename(columns={'is_attributed': 'mean2'}).reset_index()    
    df_train = pd.merge(df_train, df2, on=['app', 'channel'], how='left')
    df_val = pd.merge(df_val, df2, on=['app', 'channel'], how='left')
    #df_test = pd.merge(df_test, df2, on=['app', 'channel'], how='left')
                   
    df3 = df_train[['is_attributed', 'app', 'device']].groupby(['app', 'device']).median().rename(columns={'is_attributed': 'mean3'}).reset_index()    
    df_train = pd.merge(df_train, df3, on=['app', 'device'], how='left')
    df_val = pd.merge(df_val, df3, on=['app', 'device'], how='left')
    #df_test = pd.merge(df_test, df3, on=['app', 'device'], how='left')    
    return df_train, df_val #, df_test


In [7]:
sample = pd.read_csv('input/train_sample.csv', dtype=dtypes, parse_dates=['click_time', 'attributed_time'])
sample['click_hour'] = sample.click_time.dt.hour

print(sample.columns.values)
print(sample.shape)


['ip' 'app' 'device' 'os' 'channel' 'click_time' 'attributed_time'
 'is_attributed' 'click_hour']
(100000, 9)


In [8]:
import keras
from keras.models import Model
from keras.layers import *
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.engine.topology import Layer
import keras.backend as K

EMBEDS = [
    {'name': 'ip', 'dim': [1024, 100]},
    {'name': 'app', 'dim': [1024, 100]},
    {'name': 'device', 'dim': [1024, 50]},
    {'name': 'os', 'dim': [1024, 50]},
    {'name': 'channel', 'dim': [1024, 50]},
    {'name': 'click_hour', 'dim': [24, 5]}
]

def embed(input_dim, output_dim, x):
    e = Embedding(input_dim, output_dim, input_length=1, embeddings_regularizer=l2(1e-8))
    r = e(x)
    r = Reshape((output_dim,))(r)
    return r

def build_model(features):
    misc = Input(shape=(features,), name='misc')
    
    for e in EMBEDS:
        e['input'] = Input(shape=(1,), name=e['name'])
        e['layer'] = embed(e['dim'][0], e['dim'][1], e['input'])
        
    h = concatenate([misc] + [e['layer'] for e in EMBEDS])
    h = BatchNormalization()(h)
    
    h = Dense(32, activation='relu')(h)
    h = Dropout(0.5)(h)
    
    h = Dense(16, activation='relu')(h)
    
    h = Dense(1, activation='sigmoid')(h)
    
    model = Model(
            inputs=[misc] + [e['input'] for e in EMBEDS],
            outputs=h)
    
    #model.compile(optimizer='sgd', loss='binary_crossentropy')
    model.compile(optimizer='sgd', loss=roc_auc_score_fn)
    return model    
    
    

In [9]:
import tensorflow as tf

def roc_auc_score_fn(y_true, y_pred):
    """ ROC AUC Score.
    
    Source:
    https://github.com/tflearn/tflearn/blob/1b0e4d3539e1c4c6b31735499328c27da3a874b0/tflearn/objectives.py
    
    Approximates the Area Under Curve score, using approximation based on
    the Wilcoxon-Mann-Whitney U statistic.
    Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. (2003).
    Optimizing Classifier Performance via an Approximation to the Wilcoxon-Mann-Whitney Statistic.
    Measures overall performance for a full range of threshold levels.
    `y_pred` and `y_true` must have the same type and shape.
    """
    pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
    neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))

    pos = tf.expand_dims(pos, 0)
    neg = tf.expand_dims(neg, 1)

    # original paper suggests performance is robust to exact parameter choice
    gamma = 0.2
    p     = 3

    difference = tf.zeros_like(pos * neg) + pos - neg - gamma

    masked = tf.boolean_mask(difference, difference < 0.0)

    return tf.reduce_sum(tf.pow(-masked, p))
      

In [10]:
def hashed(s, size=10):
    return s.apply(lambda x: hash(x) % 2 ** size)

def build_input(df):
    embed_names = [e['name'] for e in EMBEDS]
    h = {'misc': df[[x for x in df.columns if x not in embed_names]] }
    h.update(dict([(name, hashed(df.loc[:, name]).values) for name in embed_names]))
    return h


In [11]:
class CustomMetric(keras.callbacks.Callback):    
    def __init__(self, val_x, val_y):
        self.val_x = val_x
        self.val_y = val_y
        
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            logs['val-auc'] = roc_auc_score(self.val_y, self.model.predict(self.val_x).ravel())
        


In [12]:

X_train = sample
y_train = sample.is_attributed.values

oof_pred = np.zeros_like(y_train)

skf = StratifiedKFold(n_splits=10)
for fold_no, (train_index, dev_index) in enumerate(skf.split(X_train, y_train)):
    X_train1, X_dev1 = X_train.iloc[train_index], X_train.iloc[dev_index]
    y_train1, y_dev1 = y_train[train_index], y_train[dev_index]
    
    #build features inside fold.
    #X_train1, X_dev1 = add_meanx2(X_train1, X_dev1)    
    
    X_train1 = X_train1.drop(['attributed_time', 'click_time', 'is_attributed'], axis=1)
    X_dev1 = X_dev1.drop(['attributed_time', 'click_time', 'is_attributed'], axis=1)
    
    
    train1_input = build_input(X_train1)
    dev_input = build_input(X_dev1)
    
    print('fold ', fold_no)
    model = build_model(train1_input['misc'].shape[1])
    
    # callbacks
    ckpt = keras.callbacks.ModelCheckpoint(filepath='tmp/weights.hdf5', 
                                           # val-auc is calculated by the custom eval_callback
                                           monitor='val-auc', mode='max',
                                           verbose=1, save_best_only=True)    
    eval_callback = CustomMetric(dev_input, y_dev1)
    
    model.fit(train1_input, y_train1, 
              validation_data=(dev_input, y_dev1),
              callbacks=[eval_callback, ckpt],
              epochs=5)
    
    model.load_weights('tmp/weights.hdf5')
    
    print('fold {}, best train-auc: {} val-auc: {}'.format(
        fold_no, 
        roc_auc_score(y_train1, model.predict(train1_input).ravel()),
        roc_auc_score(y_dev1, model.predict(dev_input).ravel())))
    
    oof_pred[dev_index] = model.predict(dev_input).ravel()

print('final val auc', roc_auc_score(y_train, oof_pred))

fold  0
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 89999 samples, validate on 10001 samples
Epoch 1/5
89999/89999 [==============================] - 14s 160us/step - loss: 0.0173 - val_loss: 0.0047
Epoch 2/5
89999/89999 [==============================] - 14s 153us/step - loss: 0.0081 - val_loss: 0.0027
Epoch 3/5
89999/89999 [==============================] - 14s 154us/step - loss: 0.0053 - val_loss: 0.0020
Epoch 4/5
89999/89999 [==============================] - 14s 153us/step - loss: 0.0029 - val_loss: 0.0016
Epoch 5/5
89999/89999 [==============================] - 14s 152us/step - loss: 0.0016 - val_loss: 0.0015
fold 0, best train-auc: 0.9885535298812436 val-auc: 0.9798033935527727
fold  1
Train on 89999 samples, validate on 10001 samples
Epoch 1/5
89999/89999 [==============================] - 14s 160us/step - loss: 0.0160 - val_loss: 0.0078
Epoch 2/5
89999/89999 [========